In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [4]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [5]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [6]:
year_data1=year_data.copy()
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [7]:
all_train_data1.iloc[:,0].mean()

22.527413194444442

In [8]:
all_train_data1.T.shape

(18, 5760)

In [9]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [10]:
for idx in [1,2,4,6,9,12,13]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >=0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
correct value between  : 1.8 1.8
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.12 

pm2.5 end ------------
correct value between  : 0.34 0.26
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 2.5 0.3
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 0.1 1.0
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -1.1 

pm2.5 end ------------
correct value between  : 6.6 1.2
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
---------

In [11]:
all_train_data1.iloc[0,:5]

0    14.00
1     1.80
2     0.51
3     0.20
4     0.90
Name: 0, dtype: float64

In [12]:
all_train_data2=all_train_data1.apply(lambda x : (x-x.mean())/x.std())

In [13]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.527413,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.290152,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,-12.300000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [14]:
all_train_data1.iloc[:,0].mean()

22.527413194444442

In [15]:
all_train_data2.iloc[0:10,0]

0   -1.355677
1   -1.355677
2   -1.355677
3   -1.514655
4   -1.673634
5   -1.673634
6   -1.673634
7   -1.673634
8   -1.196698
9   -0.878741
Name: 0, dtype: float64

In [16]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,2.921351e-15,-4.132630e-13,-1.393800e-14,5.078230e-15,3.474738e-15,-2.338465e-15,-3.438183e-15,-4.431640e-16,2.485088e-16,1.855094e-16,4.130921e-15,-9.199431e-16,5.026265e-16,3.205650e-13,8.574353e-17,5.245418e-16,5.851993e-16,1.040902e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.536816e+00,-1.518692e+01,-1.571093e+00,-1.341940e+00,-1.417839e+00,-1.636509e+00,-1.933149e+00,-1.705857e+00,-1.628736e+00,-1.345187e+00,-9.808389e-02,-3.310232e+00,-2.401358e+00,-1.121682e+01,-1.631707e+00,-1.673570e+00,-2.156207e+00,-1.611732e+00
25%,-7.197621e-01,-8.174309e-01,-4.276098e-01,-6.730107e-01,-4.538382e-01,-6.829821e-01,-6.529812e-01,-7.434694e-01,-7.134846e-01,-7.450388e-01,-9.808389e-02,-6.907360e-01,-6.401558e-01,-7.680033e-01,-8.807613e-01,-8.710141e-01,-7.482952e-01,-7.648192e-01
50%,7.513122e-02,-1.912605e-02,-1.494652e-01,-2.907653e-01,-2.347470e-01,-2.466224e-01,-2.966459e-01,-1.553437e-01,-1.795877e-01,-2.049049e-01,-9.808389e-02,1.325340e-01,-2.548928e-01,-2.180656e-01,-3.898786e-01,-3.905364e-01,-1.851306e-01,-2.002106e-01
75%,7.110459e-01,7.791788e-01,1.904894e-01,3.781641e-01,7.198057e-02,4.644824e-01,3.632342e-01,5.397139e-01,5.449866e-01,4.702623e-01,-9.808389e-02,8.061186e-01,3.505206e-01,3.318721e-01,5.918869e-01,6.390586e-01,5.657555e-01,5.526008e-01
max,2.141854e+00,2.375788e+00,2.219482e+01,1.108104e+01,1.264781e+01,5.797768e+00,7.753892e+00,1.064478e+01,5.273788e+00,5.436493e+00,3.607990e+01,1.928760e+00,1.058751e+01,6.381187e+00,2.127201e+00,2.128011e+00,5.071072e+00,4.975368e+00


In [17]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LassoCV,RidgeCV,ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

In [18]:
x=np.empty((471*12,18*9))
y=np.empty((471*12,1))

In [19]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=all_train_data2.iloc[m*480+g:m*480+g+9,:].values.reshape(1,-1)
        y[m*471+g:g*471+g+1,:]=all_train_data2.iloc[m*480+g+9:m*480+g+9+1,9].values.reshape(1,1)

In [20]:
x[9,9:11]

array([ 0.5152735 , -0.09808389])

In [21]:
y[0:2]

array([[0.5152735 ],
       [1.17543706]])

In [22]:
all_train_data2.iloc[10:11,9]

10    1.175437
Name: 9, dtype: float64

In [23]:
x[1,-10:]

array([-0.17958771,  0.5152735 , -0.09808389, -1.28947789,  6.73487971,
        0.88180982, -1.15231349, -0.05789803, -1.40532054, -1.32942776])

In [24]:
y[0]

array([0.5152735])

In [25]:
kf=KFold(n_splits=10,shuffle=True,random_state=42)

def rmse(y,y_hat):
    return np.sqrt(mean_squared_error(y,y_hat))

def cv_rmse(model,x=x,y=y):
    rmse=np.sqrt( -cross_val_score(model,x,y,scoring='neg_mean_squared_error',cv=kf) )
    return rmse

In [26]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [27]:
ridge=make_pipeline(RobustScaler(),RidgeCV(alphas=alphas_alt,cv=kf))
lasso=make_pipeline(RobustScaler(),LassoCV(max_iter=1e7,alphas=alphas2,random_state=42,cv=kf))
elasticnet=make_pipeline(RobustScaler(),ElasticNetCV(max_iter=1e7,alphas=e_alphas,cv=kf,l1_ratio=e_l1ratio))

In [28]:
score=cv_rmse(ridge)
print("Ridge: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge: 0.3776 (0.0248)



In [29]:
score = cv_rmse(lasso)
print("LASSO: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d arr

LASSO: 0.3759 (0.0246)



In [30]:
score = cv_rmse(elasticnet)
print("elastic net: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d arr

elastic net: 0.3760 (0.0247)



#### LSTM

In [31]:
class LSTM_Reg(nn.Module):
    def __init__(self,input_size,hidden_size,layer_size,output_size):
        super().__init__()
        self.lstm_net=nn.LSTM(input_size=input_size,hidden_size=hidden_size,num_layers=layer_size)
        self.dense=nn.Linear(hidden_size,output_size)
        self.state=None
    
    def forward(self,x,state):
        y,self.state=self.lstm_net(x,state)
        output=self.dense(y.view(-1,y.shape[-1]))
        return output,self.state
    

In [42]:
x9=torch.Tensor(x)
y9=torch.Tensor(y)

In [49]:
a=torch.utils.data.TensorDataset(x9,y9)
for x2,y2 in  torch.utils.data.DataLoader(a,32,shuffle=True):
    print(x2.shape)
    print(y2.shape)
    print(x2.unsqueeze(dim=0).shape)

torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 1

In [69]:
def train_pytorch(num_epochs,lr,train_features,train_labels,batch_size):
    net=LSTM_Reg(162,256,2,1)
    loss=nn.MSELoss()
    optimizer=optim.Adam(net.parameters(),lr=5e-3)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True,drop_last=True)
    state=None
    for e in range(num_epochs):
        l_sum=[]
        net.train()
        for x,y in data_iter:
            if state is not None:
                if isinstance(state ,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
                
            x=x.unsqueeze(dim=0)
            (output,state)=net(x,state)
            l=loss(output,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            l_sum.append(l.item())
            
        if (e+1) % 50 ==0:
            net.eval()
            with torch.no_grad():
                print('epoch %d , train loss mean : %f,train loss std : %f' % ( e+1,np.mean(l_sum),np.std(l_sum) ))

In [70]:
train_pytorch(1000,0.001,x9,y9,32)

epoch 50 , train loss mean : 0.028820,train loss std : 0.014839
epoch 100 , train loss mean : 0.013204,train loss std : 0.004687
epoch 150 , train loss mean : 0.009324,train loss std : 0.003893
epoch 200 , train loss mean : 0.006715,train loss std : 0.002507
epoch 250 , train loss mean : 0.004740,train loss std : 0.001726
epoch 300 , train loss mean : 0.003913,train loss std : 0.001512
epoch 350 , train loss mean : 0.003096,train loss std : 0.001465
epoch 400 , train loss mean : 0.003062,train loss std : 0.001332
epoch 450 , train loss mean : 0.002506,train loss std : 0.000976
epoch 500 , train loss mean : 0.002261,train loss std : 0.001072
epoch 550 , train loss mean : 0.001851,train loss std : 0.000735
epoch 600 , train loss mean : 0.001933,train loss std : 0.000912
epoch 650 , train loss mean : 0.001914,train loss std : 0.000946
epoch 700 , train loss mean : 0.001746,train loss std : 0.000905
epoch 750 , train loss mean : 0.001509,train loss std : 0.000816
epoch 800 , train loss mea

In [56]:
471*12

5652

### compared with linear model

In [77]:
def train_pytorch_LInear(num_epochs,lr,train_features,train_labels,batch_size):
    net=nn.Sequential(nn.Linear(train_features.shape[1],128),nn.ReLU(),nn.Linear(128,1))
    loss=nn.MSELoss()
    optimizer=optim.Adam(net.parameters(),lr=lr)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for e in range(num_epochs):
        l_sum=[]
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum.append(l.item())
        if (e+1) % 50 ==0:
            net.eval()
            with torch.no_grad():
                print('epoch %d , train loss mean : %f,train loss std : %f' % ( e+1,np.mean(l_sum),np.std(l_sum) ))

In [1]:
train_pytorch_LInear(1000,0.001,x9,y9,32)

NameError: name 'train_pytorch_LInear' is not defined